In [1]:
# Translated to .py by Yundi Zhang
# Jan 05 2017
# Adapted to PandasBiogeme by Michel Bierlaire
# Sun Nov  4 14:07:41 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.models import loglogit

pandas = pd.read_table("netherlands.dat")
database = db.Database("netherlands",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)
from biogeme.expressions import *


#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_SP_CAR	 = Beta('ASC_SP_CAR',0,None,None,1)
ASC_SP_RAIL	 = Beta('ASC_SP_RAIL',0,None,None,0)
ASC_RP_CAR	 = Beta('ASC_RP_CAR',0,None,None,1)
ASC_RP_RAIL	 = Beta('ASC_RP_RAIL',0,None,None,0)
BETA_COST	 = Beta('BETA_COST',0,None,None,0)
BETA_TIME	 = Beta('BETA_TIME',0,None,None,0)
BETA_INERT	 = Beta('BETA_INERT',0,None,None,0)
THETA_SP	 = Beta('THETA_SP',1,0,None,0)

# Define here arithmetic expressions for name that are not directly available from the data
rail_time  = DefineVariable('rail_time', rail_ivtt   +  rp_rail_ovt ,database)
car_time  = DefineVariable('car_time', car_ivtt   +  rp_car_ovt  ,database)

# Utilities
SP__Car = ( ASC_SP_CAR + BETA_COST * car_cost + BETA_TIME * car_time ) * THETA_SP
SP__Rail = ( ASC_SP_RAIL + BETA_COST * rail_cost + BETA_TIME * rail_time + BETA_INERT * rp_choice ) * THETA_SP
RP__Car =  ASC_RP_CAR + BETA_COST * car_cost + BETA_TIME * car_time
RP__Rail =  ASC_RP_RAIL + BETA_COST * rail_cost + BETA_TIME * rail_time 
V = {0: RP__Car,1: RP__Rail,10: SP__Car,11: SP__Rail}
av = {0: rp,1: rp,10: sp,11: sp}

# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "RP-SP_NL_rpsp"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

             Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
ASC_RP_RAIL  0.798     0.27    2.96  0.00311         0.275          2.9   
ASC_SP_RAIL  -4.79     1.35   -3.54 0.000404          1.35        -3.54   
BETA_COST    -0.05  0.00937   -5.34 9.22e-08       0.00966        -5.18   
BETA_INERT    8.04      1.9    4.24 2.26e-05          1.91          4.2   
BETA_TIME    -1.32    0.288   -4.59 4.53e-06         0.294        -4.51   
THETA_SP     0.339   0.0815    4.16 3.24e-05        0.0817         4.15   

             Rob. p-value  
ASC_RP_RAIL       0.00373  
ASC_SP_RAIL      0.000401  
BETA_COST         2.2e-07  
BETA_INERT       2.63e-05  
BETA_TIME        6.62e-06  
THETA_SP         3.36e-05  
Nbr of observations: 1739
LL(0) =    -1205.383
LL(beta) = -780.124
rho bar square = 0.348
Output file: RP-SP_NL_rpsp.html


In [2]:
pandasResults

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_RP_RAIL,0.798,0.27,2.96,0.00311,0.275,2.9,0.00373
ASC_SP_RAIL,-4.79,1.35,-3.54,0.000404,1.35,-3.54,0.000401
BETA_COST,-0.05,0.00937,-5.34,9.22e-08,0.00966,-5.18,2.2e-07
BETA_INERT,8.04,1.9,4.24,2.26e-05,1.91,4.2,2.63e-05
BETA_TIME,-1.32,0.288,-4.59,4.53e-06,0.294,-4.51,6.62e-06
THETA_SP,0.339,0.0815,4.16,3.24e-05,0.0817,4.15,3.36e-05


In [5]:
pandas

,id,rp,sp,choice,purpose,npersons,age,employ_status,mainearn,arrival_time,...,car_cost,car_walk_time,car_parking_fee,rail_comfort,rp_transfer,rp_choice,rp_rail_ovt,rp_car_ovt,rail_time,car_time
0,1,1,0,0,0,3,0,0,0,0,...,5,0.167,0,-1,0,0,0.5,0.167,2.5,1.17
1,1,0,1,10,0,3,0,0,0,0,...,7.5,-1,-1,0,0,0,0.5,0.167,2.67,1.33
2,1,0,1,10,0,3,0,0,0,0,...,7,-1,-1,0,0,0,0.5,0.167,2.67,1.33
3,1,0,1,10,0,3,0,0,0,0,...,6.5,-1,-1,0,0,0,0.5,0.167,2.67,1.33
4,1,0,1,10,0,3,0,0,0,0,...,6,-1,-1,0,0,0,0.5,0.167,2.67,1.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1734,235,0,1,10,0,4,1,1,0,1,...,7.5,-1,-1,1,2,0,0.25,0.017,1.17,1.53
1735,235,0,1,10,0,4,1,1,0,1,...,6.88,-1,-1,1,2,0,0.25,0.017,1.17,1.53
1736,235,0,1,10,0,4,1,1,0,1,...,6.25,-1,-1,1,2,0,0.25,0.017,1.17,1.53
1737,235,0,1,10,0,4,1,1,0,1,...,6.25,-1,-1,0,2,0,0.25,0.017,1.33,1.53
